# Spark MLlib
## Spark MLlib简介
### 什么是机器学习

机器学习是一门人工智能科学，该领域的主要研究对象是人工智能。机器学习利用数据或以往的经验，以此优化计算机程序的性能标准  
机器学习强调三个关键词：算法、经验、性能

### 基于大数据的机器学习

- 传统的机器学习算法，由于技术和单机存储的限制，只能在少量数据上使用，依赖数据抽样
- 大数据技术的出现，可以支持在全量数据上进行机器学习
- 机器学习算法涉及大量迭代计算
- 基于磁盘的MapReduce不适合进行大量迭代计算
- 基于内存的Spark比较适合进行大量迭代计算

### Spark机器学习库MLlib

- Spark提供了一个基于海量数据的机器学习库，它提供了常用机器学习算法的分布式实现
- 开发者只需要有Spark基础并且了解机器学习算法的原理，以及方法相关参数的含义，就可以轻松的调用相应的API来实现基于海量数据的机器学习过程
- pyspark的即席查询也是一个关键。算法工程师可以编写代码边看结果
- 需要注意的是，MLlib中只包含能够在集群上运行良好的并行算法，这一点很重要
- 一些较新的研究得出的算法因为使用于集群，也被包含在MLlib中，例如分布式随机森林算法、最小交替二乘法。
- 如果是小规模数据集上训练各机器学习模型，可以是用单节点的机器学习算法库（Weka）
- MLlib由一些通用的机器学习算法和工具组成，包括分类、回归、聚类、协同过滤、降维等,同时还包括底层的优化原语和高层的流水线（Pipeline）API，具体如下：
    - 算法工具：常用的学习算法，如分类、回归、聚类和协同过滤
    - 特征化工具：特征提取、转化、降维和选择工具
    - 流水线(Pipeline)：用于构建、评估和调整机器学习工作流的工具
    - 持久性：保存和加载算法、模型和管道
    - 实用工具：线性代数、统计、数据处理等工具
    
    
    
MLlib目前支持4中常见的机器学习问题：分类、回归、聚类和协同过滤
    
    
  ||离散数据|连续数据|  
  |--|--|--| 
  |监督学习|Classification、LogisticRegression(with Elastic-Net)、SVM、DecisionTree、RandomForest、GBT、NaiveBayes、MultilayerPerceptron、OneVsRest| Regression、LogisticRegression(with Elastic-Net)、DecsionTree、RandomFores、GBT、AFTSurvivalRegression、IsotonicRegression|
  |无监督学习|Clustering、KMeans、GuassianMixture、LDA、PowerIterationClustering、BisectingKMeans| Dimensionality Reduction、 matrix factorization、PCA、SVD、ALS、WLS|
    


## 机器学习流水线
### 机器学习流水线概念
- DataFrame：使用Spark SQL中的DataFrame作为数据集，它可以容纳各种数据类型。较之RDD，DataFrame包含了schema信息，更类似传统数据库的二维表格。
- 它被ML Pipeline用来存储源数据。例如，DataFrame中的列可以是存储的文本、特征向量、真实标签和预测的标签等

- Transformer：转换器，是一种可以将一个DataFrame转换为另一个DataFrame的算法。比如一个模型就是一个Transformer。它可以把一个不包含预测标签等测试数据集DataFrame打上标签，转换成另一个包含预测标签的DataFrame
- Estimator：估计器或评估器，它是学习算法或在训练数据上的训练方法的概念抽象。在Pipeline里通常是被用来操作DataFrame数据并生成一个Transformer。从技术上讲，Estimator实现了一个方法fit()，它接受一个DataFrame并产生一个转换器。比如一个随机森林算法就是一个Estimator，它可以调用fit()，通过训练特征数据而得到一个随机深林算法
- Parameter：Parameter被用来设置Transformer或者Estimator的参数。现在，所有转换器和评估器可共享用于指定参数的公共API。ParamMap是一组（参数，值）对
- PipeLine：流水线或者管道。流水线将多个工作流阶段（转换器和评估器）连接在一起，形成机器学习的工作流，并获得结果输出。


### 流水线工作流程
要构建一个Pipeline流水线，首先需要定义Pipeline中的各个流水线阶段PipelineStage(包括转换器和评估器），比如指标提取和转换模型训练等。有了这些处理特定问题的转换器和评估器，就可以按照具体的处理逻辑有序地组织PipelineStages并创建一个Pipeline
```python
pipeline = Pipeline(stages=[stage1, stage2, stage3]
```

然后就可以把训练数据集作为输入参数，调用Pipeline实例的fit方法来开始以流的方式来处理源训练数据。这个调用会返回一个PipelineModel类实例，进而被用来预测测试数据的标签

- 流水线的各个阶段按顺序运行，输入的DataFrame在它通过每个阶段时被转换

值得注意的是，流水线本身也可以看作是一个评估器，在流水线的fit()方法运行之后，它产生一个PipelineModel，它是一个Transformer。这个管道模型将在测试数据的时候使用

### 构建一个机器学习流水线
以逻辑斯蒂回归为例，构建一个典型的机器学习过程

**任务描述**  
查找出所有包含“spark”的句子，将包含“spark”的句子的标签设为1，没有“spark”的句子的标签设为0
- 需要使用SparkSession对象


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('WordCount').getOrCreate()

- 引入要包含的包并构建数据集

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

#Prepare training documents from a list of (id, text, label) tuples
training = spark.createDataFrame([
    (0, 'a b c d e spark', 1.0),
    (1, 'b d', 0.0),
    (2, 'spark f g h', 1.0),
    (3, 'hadoop mapreduce', 0.0),
], ['id', 'text', 'label'])

/home/spark/miniconda3/envs/bigdata/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


- 定义Pipeline中的各个流水线阶段PipelineStage，包括转换器和评估器，具体地，包含tokenizer，hashing TF和lr

In [3]:
tokenizer = Tokenizer(inputCol='text', outputCol='words')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features')
lr = LogisticRegression(maxIter=10, regParam=0.001)

- 按照具体的处理逻辑有序地组织PipelineStages，并创建一个Pipeline

In [4]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

现在构建的Pipeline本质上是一个Estimator，在它的fit()方法运行之后，它将产生一个PipelineModel，它是一个Transformer

In [5]:
model = pipeline.fit(training)

可以看到，model的类型是一个PipelineModel，这个流水线模型将在测试数据的时候使用


- 构建测试数据

In [6]:
test = spark.createDataFrame([
    (4, 'spark i j k'),
    (5, 'I m n'),
    (6, 'spark hadoop spark'),
    (7, 'apache hadoop')
], ['id', 'text'])

- 调用之前训练好的PipelineModel的transform()方法，让测试数据按顺序通过拟合的流水线，生成预测结果

In [8]:
prediction = model.transform(test)
selected = prediction.select('id', 'text', 'probability', 'prediction')
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.1596407738787475,0.8403592261212525], prediction=1.000000
(5, I m n) --> prob=[0.8378325685476744,0.16216743145232562], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976037,0.9307336686702395], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444218,0.01784246665557808], prediction=0.000000


## 特征提取和转换
### 特征提取

- “词频-逆向文件频率”(TF-IDF)是一种在文本挖掘中广泛使用的特征向量化方法，它可以体现一个文档中词语在语料库中的重要程度
- 词语由t表示，文档由d表示，语料库由D表示。词频TF(t, d)是词语t在文档d中出现的次数。文件频率DF(t, D)是包含词语的文档的个数
- TF-IDF就是在数值化文档信息，衡量词语能够提供多少信息以区分文档。其定义如下:
$$
        IDF(t,D) = log \frac {|D| + 1}{DF(t,D)+1}\

        TFIDF(t,d,D) = TF(t,d)·IDF(t,D)
$$

在Spark ML库中，TF-IDF被分成两部分：
- TF(+hashing)
    HashingTF是一个Transformer，在文本处理中，接收词条的集合然后把这些集合转化为固定长度的特征向量。这个算法在哈希的同时会统计各个词条的词频。
        
- IDF
    IDF是一个Estimator，在一个数据集上应用它的fit()方法，产生一个IDFModel。该IDFModel接收特征向量（由HashingTF产生），然后计算每一个词在文档中出现的频次。IDF会减少那些在语料库中出现频率较高的词的权重。

**过程描述**
- 在下面的代码段中，我们以一组句子开始
- 首先使用分解器Tokenizer把句子划分为单个词语
- 对每一个句子（词带），使用HashingTF将句子转换为特征向量
- 最后使用IDF重新调整特征向量（这种转换通常可以提高使用文本特征的性能）
    
    

(1)导入TF-IDF所需要的包

In [9]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

(2)创建一个简单的DataFrame，每一个句子代表一个文档

In [10]:
sentenceData = spark.createDataFrame([
    (0, 'I heard about Spark and I love Spark'),
    (0, 'I wish Java could use case classes'),
    (1, 'Logistic regression models are neat')
]).toDF('label', 'sentence')

(3)得到文档集合后，即可用tokenizer对句子进行分词

In [11]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|I heard about Spa...|[i, heard, about,...|
|    0|I wish Java could...|[i, wish, java, c...|
|    1|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



(4)得到分词后的文档序列后，即可使用HashingTF的transform()方法把句子哈希成特征向量，这里设置哈希表的桶数为2000

In [12]:
hashingTF = HashingTF(inputCol='words', outputCol='rawFeatures', numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)
featurizedData.select('words', 'rawFeatures').show(truncate=False)

+---------------------------------------------+---------------------------------------------------------------------+
|words                                        |rawFeatures                                                          |
+---------------------------------------------+---------------------------------------------------------------------+
|[i, heard, about, spark, and, i, love, spark]|(2000,[240,333,1105,1329,1357,1777],[1.0,1.0,2.0,2.0,1.0,1.0])       |
|[i, wish, java, could, use, case, classes]   |(2000,[213,342,489,495,1329,1809,1967],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[logistic, regression, models, are, neat]    |(2000,[286,695,1138,1193,1604],[1.0,1.0,1.0,1.0,1.0])                |
+---------------------------------------------+---------------------------------------------------------------------+



(5)调用IDF方法来构造特征向量的规模，生成的变量idf是一个评估器，在特征向量上应用它的fit()方法，会产生一个IDFModel（名称为idfModel）

In [13]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idfModel = idf.fit(featurizedData)

(6)调用IDFModel的transform()方法，可以得到每一个单词对应的TF-IDF度量值

In [14]:
rescaledData = idfModel.transform(featurizedData)
rescaledData.select('features', 'label').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                       |label|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(2000,[240,333,1105,1329,1357,1777],[0.6931471805599453,0.6931471805599453,1.3862943611198906,0.5753641449035617,0.6931471805599453,0.6931471805599453])                       |0    |
|(2000,[213,342,489,495,1329,1809,1967],[0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453])|0    |
|(2000,[286,695,1138,1193,1604],[0.6931471805599453,0.6931471805599453,0.6931471

- 在机器学习处理过程中，为了方便相关算法的实现，经常需要把标签数据（一般是字符串）转化成整数索引，或是在计算结束后将整数索引还原为相应的标签
- Spark ML包中提供了几个相关的转换器，例如，StringIndexer、IndexToString、OneHotEncoder、VectorIndexer，它们提供了十分方便的特征转换功能，这些转换器都位于org.apache.spark.ml.feature包下
- 值得注意的是，用于特征转换的转换器和其他的机器学习算法一样，也属于ML Pipeline模型的一部分，可以用来构成机器学习流水线，以StringIndexer为例，其存储着进行标签数值化过程的相关超参数，是一个Estimator，对其调用fit()方法即可生成相应的模型StringIndexModel类，很显然，它存储于用于DataFrame进行相关处理的参数，是一个Transformer